https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/

Semantic chunker, metni sabit bir yığın boyutuyla yığınlamak yerine, gömme benzerliğini kullanarak cümleler arasındaki kesme noktasını uyarlamalı olarak seçer. Bu, bir "chunk" 'ın anlamsal olarak birbiriyle ilişkili cümleler içermesini sağlar.

# Setup Data

In [1]:
from PyPDF2 import PdfReader

# Bu fonksiyon, belirtilen PDF dosyasını açar, her sayfadaki metni çıkarır ve tüm metni birleştirerek tek bir metin dizesi olarak döndürür.
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


pdf_path = "documents/PDF.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [2]:
def write_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

pdf_path = "documents/PDF.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

output_file_path = "documents/PDF.txt"
write_text_to_file(pdf_text, output_file_path)


In [3]:
! pip install llama-index-readers-file

In [4]:
text = """Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi yapmak veya Versailles Bahçeleri'nde ay ışığında dolaşmak, Paris'in aşk dolu atmosferini keşfetmenin sadece birkaç yoludur. Tüm bu özellikler bir araya geldiğinde, Paris'in gerçekten de Fransa'nın kalbi olduğu açıkça ortaya çıkar.
"""

def write_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

output_file_path = "documents/fransa.txt"
write_text_to_file(text, output_file_path)


In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["documents/PDF.txt"]).load_data()

c:\Users\havva\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
documents2 = SimpleDirectoryReader(input_files=["documents/fransa.txt"]).load_data()

# Define Semantic Splitter With Hugging Face

In [7]:
! pip install llama-index-embeddings-huggingface

In [8]:
! pip install llama_index.embeddings.huggingface

In [9]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Compare 

In [26]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512, chunk_overlap = 0)

In [27]:
nodes = splitter.get_nodes_from_documents(documents)
print("chunk: ", len(nodes))
print("\n")
print(nodes[0].get_content())

chunk:  49


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [29]:
nodes_text = splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(nodes_text))
print("\n")
print(nodes_text[0].get_content())

chunk:  2


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. 


baseline splitter

In [28]:
base_nodes = base_splitter.get_nodes_from_documents(documents)
print("chunk: ", len(base_nodes))
print("\n")
print(base_nodes[0].get_content())

chunk:  75


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [30]:
base_nodes_text = base_splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(base_nodes_text))
print("\n")
print(base_nodes_text[0].get_content())

chunk:  1


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi 

# Change parameters 1  / text'i daha iyi bir şekilde 2 chunk'a böldü, pdf'de 49 hunk elde etti

- buffer_size=1: Bu parametre, işlenmeden önce bellekte tutulan düğüm sayısını kontrol eder. Bunu 1'e ayarlamak, bölücünün her düğümü belleğe almadan bağımsız olarak işleyeceği anlamına gelir.

- breakpoint_percentile_threshold=95: Bu parametre, muhtemelen bölücünün metin düğümleri içinde anlamsal sınırları nasıl belirlediğini etkiler. 95'lik bir değer, bir kesme noktasının (bölme noktası), ardışık metin segmentleri arasındaki gömüm mesafesi tüm gömüm mesafelerinin 95. persentilini aştığında ortaya çıkabileceğini önerir. Bu, düğümü anlamsal olarak farklı bölümlerde bölmeye yardımcı olur.

- embed_model=embed_model: Bu parametre, daha önce oluşturulan "embed_model" örneğini "SemanticSplitterNodeParser"'a aktarır. Bu, bölücünün gömüm modelini kullanarak gömüm mesafelerini hesaplamasına ve metin düğümleri içinde anlamsal kopmaları belirlemesine olanak tanır.


In [31]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=95, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512, chunk_overlap = 0)

In [32]:
nodes = splitter.get_nodes_from_documents(documents)
print("chunk: ", len(nodes))
print("\n")
print(nodes[0].get_content())

chunk:  49


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [33]:
nodes_text = splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(nodes_text))
print("\n")
print(nodes_text[0].get_content())

chunk:  2


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.




In [36]:
print(nodes_text[1].get_content())

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi yapmak veya Versailles Bahçeleri'nde ay ışığında dolaşmak, Paris'in aşk dolu atmosferini keşfetmenin sadece birkaç yoludur. Tüm bu özellikler bir araya geldiğinde, Paris'in gerçekten de Fransa'nın kalbi olduğu açıkça ortaya çıkar.



In [34]:
base_nodes = base_splitter.get_nodes_from_documents(documents)
print("chunk: ", len(base_nodes))
print("\n")
print(base_nodes[0].get_content())

chunk:  75


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [35]:
base_nodes_text = base_splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(base_nodes_text))
print("\n")
print(base_nodes_text[0].get_content())

chunk:  1


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi 

# Change parameters 2 / Hepsinden 1 chunk elde etti / OLMAZ

In [37]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=100, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=300, chunk_overlap = 0)

In [38]:
nodes = splitter.get_nodes_from_documents(documents)
print("chunk: ", len(nodes))
print("\n")
print(nodes[0].get_content())

chunk:  1


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuric

In [39]:
nodes_text = splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(nodes_text))
print("\n")
print(nodes_text[0].get_content())

chunk:  1


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi 

In [40]:
base_nodes = base_splitter.get_nodes_from_documents(documents)
print("chunk: ", len(base_nodes))
print("\n")
print(base_nodes[0].get_content())

chunk:  134


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zur

In [41]:
base_nodes_text = base_splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(base_nodes_text))
print("\n")
print(base_nodes_text[0].get_content())

chunk:  2


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır.


# Change parameters 3 / text'te 1 chunk elde etti, pdf'te 75 / bu da çok iyi değil 

In [42]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=100, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512, chunk_overlap = 0)

In [43]:
nodes = splitter.get_nodes_from_documents(documents)
print("chunk: ", len(nodes))
print("\n")
print(nodes[0].get_content())

chunk:  1


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuric

In [44]:
nodes_text = splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(nodes_text))
print("\n")
print(nodes_text[0].get_content())

chunk:  1


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi 

In [45]:
base_nodes = base_splitter.get_nodes_from_documents(documents)
print("chunk: ", len(base_nodes))
print("\n")
print(base_nodes[0].get_content())

chunk:  75


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [46]:
base_nodes_text = base_splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(base_nodes_text))
print("\n")
print(base_nodes_text[0].get_content())

chunk:  1


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi 

# Change parameters 4 / text'i yine daha iyi bir şekilde 2 ye böldü ,pdf'de 49 chunk elde etti

In [47]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=3, breakpoint_percentile_threshold=95, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512, chunk_overlap = 0)

In [48]:
nodes = splitter.get_nodes_from_documents(documents)
print("chunk: ", len(nodes))
print("\n")
print(nodes[0].get_content())

chunk:  49


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [49]:
nodes_text = splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(nodes_text))
print("\n")
print(nodes_text[0].get_content())

chunk:  2


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.




In [50]:
base_nodes = base_splitter.get_nodes_from_documents(documents)
print("chunk: ", len(base_nodes))
print("\n")
print(base_nodes[0].get_content())

chunk:  75


Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zuri

In [51]:
base_nodes_text = base_splitter.get_nodes_from_documents(documents2)
print("chunk: ", len(base_nodes_text))
print("\n")
print(base_nodes_text[0].get_content())

chunk:  1


Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi 

# Mixed parameters and texts

In [1]:
! pip install llama-index-embeddings-huggingface

In [4]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [5]:
text = """
Geleceğin İş Alanları: Yükselen Kariyer Fırsatları ve Yetenekler
Giriş:
Teknoloji ve küreselleşme, iş dünyasını sürekli olarak dönüştürmekte ve yeni kariyer fırsatları doğurmaktadır. Bu yazıda, geleceğin iş alanlarını ve bu alanlarda önemli olan yetenekleri ele alacağız.
Yapay Zeka ve Makine Öğrenimi:
Yapay zeka ve makine öğrenimi, hızla gelişen alanlardan biridir. Şirketler, veri analizi, otomasyon ve öngörüsel analizler için yapay zeka tabanlı çözümlere yönelmektedir. Bu alanda uzmanlaşmış yetenekler, önümüzdeki yıllarda büyük talep görecektir.
Sürdürülebilirlik ve Yeşil Teknolojiler:
Çevre dostu teknolojilerin ve sürdürülebilir iş uygulamalarının önemi giderek artmaktadır. Yeşil enerji, geri dönüşüm, çevresel etki analizi gibi alanlarda uzmanlaşan profesyoneller, sürdürülebilirlik odaklı işlerde önemli roller üstlenecektir.
Uzaktan Çalışma ve Dijital Yönetim:
Pandemi süreciyle birlikte uzaktan çalışma ve dijital iş modelleri hızla yaygınlaşmıştır. Uzaktan çalışma becerileri ve dijital iletişim yetenekleri, geleceğin iş dünyasında vazgeçilmez olacaktır.
Veri Güvenliği ve Siber Güvenlik:
Dijitalleşme arttıkça veri güvenliği endişeleri de artmaktadır. Siber güvenlik uzmanları, veri ihlallerini önlemek ve şirketlerin dijital varlıklarını korumak için önemli bir rol oynayacaktır.
Sonuç:
Geleceğin iş alanları, teknolojinin hızlı evrimi ve küresel trendler tarafından şekillendirilmektedir. Bu yazıda ele aldığımız alanlar, iş arayanlar ve kariyerlerini geliştirmek isteyenler için önemli fırsatlar sunmaktadır.


Stres Yönetimi: Sağlıklı ve Mutlu Bir Yaşam İçin İpuçları"
Giriş:
Modern yaşamın getirdiği yoğun iş temposu, kişisel sorumluluklar ve belirsizlikler stres seviyelerini artırmaktadır. Bu yazıda, stres yönetimi konusunda sağlıklı ve mutlu bir yaşam için bazı ipuçlarını ele alacağız.
Fiziksel Aktivite ve Spor:
Düzenli egzersiz yapmak, stresi azaltmanın etkili bir yoludur. Spor yapmak, endorfin salgılanmasını artırır ve ruh halini iyileştirir. Haftada birkaç kez egzersiz yapmak, stresle başa çıkmak için önemli bir stratejidir.
Zihinsel Dinlenme ve Meditasyon:
Meditasyon ve derin nefes almak, zihni sakinleştirir ve stresi azaltır. Günlük meditasyon uygulamaları, zihinsel dinginlik sağlayarak stresle baş etmeyi kolaylaştırır.
Sosyal Bağlantılar ve Destek:
Aile ve arkadaşlarla zaman geçirmek, sosyal destek ağını güçlendirmek stresle mücadelede önemlidir. Zor zamanlarda başkalarından yardım istemek ve duygusal destek almak, stresi azaltmaya yardımcı olabilir.
Zaman Yönetimi ve Hedef Belirleme:
Zamanı etkili bir şekilde yönetmek, işleri düzenlemek ve öncelikleri belirlemek stresi azaltabilir. Hedef belirleme ve adım adım ilerleme kaydetme, kontrol duygusunu artırarak stresle başa çıkmayı kolaylaştırır.
Sonuç:
Stres, modern yaşamın bir gerçeğidir ancak etkili stres yönetimi stratejileriyle başa çıkılabilir. Bu yazıda ele aldığımız ipuçları, sağlıklı ve mutlu bir yaşam için stresi azaltmanın yollarını sunmaktadır.

"""

def write_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

output_file_path = "documents/text3.txt"
write_text_to_file(text, output_file_path)


In [6]:
! pip install llama-index-readers-file

In [37]:
from llama_index.core import SimpleDirectoryReader
text = SimpleDirectoryReader(input_files=["documents/text3.txt"]).load_data()

In [57]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=90, embed_model=embed_model
)

In [59]:
nodes_text = splitter.get_nodes_from_documents(text)
print("chunk: ", len(nodes_text))
print("\n")
print(nodes_text[0].get_content())

chunk:  9


Geleceğin İş Alanları: Yükselen Kariyer Fırsatları ve Yetenekler
Giriş:
Teknoloji ve küreselleşme, iş dünyasını sürekli olarak dönüştürmekte ve yeni kariyer fırsatları doğurmaktadır. Bu yazıda, geleceğin iş alanlarını ve bu alanlarda önemli olan yetenekleri ele alacağız.
Yapay Zeka ve Makine Öğrenimi:
Yapay zeka ve makine öğrenimi, hızla gelişen alanlardan biridir. Şirketler, veri analizi, otomasyon ve öngörüsel analizler için yapay zeka tabanlı çözümlere yönelmektedir. 


In [60]:
print(nodes_text[1].get_content())

Bu alanda uzmanlaşmış yetenekler, önümüzdeki yıllarda büyük talep görecektir.
Sürdürülebilirlik ve Yeşil Teknolojiler:
Çevre dostu teknolojilerin ve sürdürülebilir iş uygulamalarının önemi giderek artmaktadır. Yeşil enerji, geri dönüşüm, çevresel etki analizi gibi alanlarda uzmanlaşan profesyoneller, sürdürülebilirlik odaklı işlerde önemli roller üstlenecektir.
Uzaktan Çalışma ve Dijital Yönetim:
Pandemi süreciyle birlikte uzaktan çalışma ve dijital iş modelleri hızla yaygınlaşmıştır. Uzaktan çalışma becerileri ve dijital iletişim yetenekleri, geleceğin iş dünyasında vazgeçilmez olacaktır.
Veri Güvenliği ve Siber Güvenlik:
Dijitalleşme arttıkça veri güvenliği endişeleri de artmaktadır. Siber güvenlik uzmanları, veri ihlallerini önlemek ve şirketlerin dijital varlıklarını korumak için önemli bir rol oynayacaktır.
Sonuç:
Geleceğin iş alanları, teknolojinin hızlı evrimi ve küresel trendler tarafından şekillendirilmektedir. Bu yazıda ele aldığımız alanlar, iş arayanlar ve kariyerlerini gel

In [161]:
print(nodes_text[2].get_content())

Büyük bir savaş başladı. Işık ve gölge, iyilik ve kötülük arasında bir mücadele yaşandı. Luminar'ın cesareti ve kararlılığı, Malakar'ın karanlık güçlerini alt etti. Güneş, tekrar ormanın üzerine doğduğunda, krallık kurtarılmıştı. Barış ve huzur yeniden sağlandı.



In [162]:
print(nodes_text[3].get_content())

Luminar ve dostları, krallığın kahramanları olarak anıldı. Onların cesareti ve fedakarlığı, efsaneleşti. Krallık, bir kez daha eski ihtişamına kavuştu, ve halkı mutlu bir geleceğe doğru ilerledi. Ve böylece, Bilgelik Ormanı'nda yaşanan bu destansı macera, sonsuza dek unutulmayacak bir hikaye olarak kaldı.
Fakat krallık, her zaman yeni maceralara açıktı. Belki de bir sonraki tehdit, daha da büyük ve karanlık olacaktı. Ancak bir şey kesindi: Krallığın halkı, birlik ve dayanışma içinde olduğu sürece, hiçbir karanlık güç onları yıkamazdı.
Gece, şehrin sokaklarını saran bir sessizlikle başladı. Yıldızlar, gökyüzünde parıldarken, ayın loş ışığı sokak lambalarının altında dans ediyordu. 


In [147]:
print(nodes_text[4].get_content())

Luminar ve dostları, krallığın kahramanları olarak anıldı. Onların cesareti ve fedakarlığı, efsaneleşti. Krallık, bir kez daha eski ihtişamına kavuştu, ve halkı mutlu bir geleceğe doğru ilerledi. 


In [149]:
print(nodes_text[6].get_content())

Saatler ilerledikçe, bilgisayarının ekranında yeni bir yazılım parçası belirmeye başladı. Her satırı yazarken, heyecanı ve tutkusu artıyordu.
Saatler geçti, ancak David'in enerjisi hiç azalmadı. Sabahın erken saatlerine kadar çalıştı ve sonunda projenin ilk versiyonunu tamamladı. Yorgun ama mutlu bir şekilde, bilgisayarının başından kalktı ve yatağına doğru ilerledi. Yarın, bu yeni projenin detaylarını ekibine anlatacaktı.
Şehir uyurken, David'in apartman dairesi sessizliğe gömüldü. Ancak o, yeni başarılarına doğru bir adım daha atmıştı. Gelecekte, bu geceyi bir dönüm noktası olarak hatırlayacak ve yeni projelerine daha da büyük bir hevesle sarılacaktı.




In [92]:
# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=300, chunk_overlap = 0)

In [93]:
base_nodes_text = base_splitter.get_nodes_from_documents(text)
print("chunk: ", len(base_nodes_text))
print("\n")
print(base_nodes_text[0].get_content())

chunk:  10


Geleceğin İş Alanları: Yükselen Kariyer Fırsatları ve Yetenekler
Giriş:
Teknoloji ve küreselleşme, iş dünyasını sürekli olarak dönüştürmekte ve yeni kariyer fırsatları doğurmaktadır. Bu yazıda, geleceğin iş alanlarını ve bu alanlarda önemli olan yetenekleri ele alacağız.
Yapay Zeka ve Makine Öğrenimi:
Yapay zeka ve makine öğrenimi, hızla gelişen alanlardan biridir. Şirketler, veri analizi, otomasyon ve öngörüsel analizler için yapay zeka tabanlı çözümlere yönelmektedir. Bu alanda uzmanlaşmış yetenekler, önümüzdeki yıllarda büyük talep görecektir.
Sürdürülebilirlik ve Yeşil Teknolojiler:
Çevre dostu teknolojilerin ve sürdürülebilir iş uygulamalarının önemi giderek artmaktadır.


In [98]:
print(base_nodes_text[4].get_content())

Sonuç:
Stres, modern yaşamın bir gerçeğidir ancak etkili stres yönetimi stratejileriyle başa çıkılabilir. Bu yazıda ele aldığımız ipuçları, sağlıklı ve mutlu bir yaşam için stresi azaltmanın yollarını sunmaktadır.


Bir zamanlar, Bilgelik Ormanı'nın derinliklerinde, büyülü bir krallık kurulmuştu. Bu krallık, doğanın güzellikleriyle çevrili, masalsı manzaralara sahip bir cennetti. Ormanın kuşları melodik şarkılarını söylerken, çiçeklerin kokusu rüzgarla taşınıyordu. Krallığın halkı, barış ve uyum içinde yaşarken, kralları adaletle hükmediyordu.
Ancak bu huzur dolu günler, karanlık bir gölgeyle kesildi. Kara büyücü Malakar, güç arzusuyla dolup taşarak krallığı ele geçirmeye kararlıydı.


In [183]:
print(base_nodes_text[].get_content())

Ancak bir şey kesindi: Krallığın halkı, birlik ve dayanışma içinde olduğu sürece, hiçbir karanlık güç onları yıkamazdı.
Gece, şehrin sokaklarını saran bir sessizlikle başladı. Yıldızlar, gökyüzünde parıldarken, ayın loş ışığı sokak lambalarının altında dans ediyordu. Rüzgar, hafifçe eserek yaprakları hışırtıyla dans ettiriyordu. Şehrin gürültüsü, günün yorgunluğuyla birlikte solmaya başlamıştı.
Bir apartman dairesinde, genç bir adam olan David, bilgisayarının başında oturmuş düşünceli bir şekilde ekrana bakıyordu. Gece yarısını çoktan geçmiş olmasına rağmen, hala çalışıyordu. Bir yazılım geliştiricisi olarak, projelerinin üzerinde saatlerce çalışması alışılagelmiş bir durumdu.
